# Проект: Маркетинг.

## Описание проекта.

Интернет-магазин собирает историю покупателей, проводит рассылки предложений и планирует будущие продажи.<br/>
Для оптимизации процессов надо выделить пользователей, которые готовы совершить покупку в ближайшее время.

### Описание данных.

**apparel-purchases история покупок**<br/><br/>
Данные о покупках клиентов по дням и по товарам. В каждой записи покупка определенного товара, его цена, количество штук.<br/>
В таблице есть списки идентификаторов, к каким категориям относится товар. Часто это вложенные категории (например автотовары-аксессуары-освежители), но также может включать в начале списка маркер распродажи или маркер женщинам/мужчинам.<br/>
Нумерация категорий сквозная для всех уровней, то есть 44 на второй позиции списка или на третьей – это одна и та же категория. Иногда дерево категорий обновляется, поэтому могут меняться вложенности, например ['4', '28', '44', '1594'] или ['4', '44', '1594']. Как обработать такие случаи – можно предлагать свои варианты решения.<br/>
- client_id идентификатор пользователя/клиента
- quantity количество товаров в заказе
- price цена товара
- category_ids вложенные категории, к которым отнсится товар (идентификаторы категорий)
- date дата покупки
- message_id идентификатор сообщения из рассылки<br/><br/>

**apparel-messages история рекламных рассылок**<br/><br/>
Рассылки, которые были отправлены клиентам из таблицы покупок.<br/>
- bulk_campaign_id идентификатор рекламной кампании (рассылки)
- client_id идентификатор пользователя/клиента
- message_id идентификатор сообщений
- event тип действия, действие с сообщением (отправлено, открыто, покупка...)
- channel канал рассылки
- date дата рассылки
- created_at точное время создания сообщения<br/><br/>

**target**<br/>
- client_id идентификатор клиента
- target клиент совершил покупку в целевом периоде<br/><br/>

**full_campaign_daily_event агрегация общей базы рассылок по дням и типам событий**<br/><br/>
Общая база рассылок огромна, поэтому собрана агрегированная по дням статистика по рассылкам.<br/>
Если будем создавать на основе этой статистики дополнительные признаки, необходимо обратить внимание, что нельзя суммировать по колонкам nunique, потому что это уникальные клиенты в пределах дня, у нас нет данных, повторяются ли они в другие дни.<br/>
- date дата
- bulk_campaign_id идентификатор рассылки
- count_event* общее количество каждого события event
- nunique_event* количество уникальных client_id в каждом событии<br/>
###### * в именах колонок найдете все типы событий event<br/><br/>

**full_campaign_daily_event_channel агрегация по дням с учетом событий и каналов рассылки**<br/>
- date дата
- bulk_campaign_id идентификатор рассылки
- count_event*_channel* общее количество каждого события по каналам
- nunique_event*_channel* количество уникальных client_id по событиям и каналам<br/>
###### * в именах колонок есть все типы событий event и каналов рассылки channel

### Предварительный план.

- Изучить данные
- Разработать полезные признаки
- Создать модель для классификации пользователей
- Улучшить модель и максимизировать метрику roc_auc
- Выполнить тестирование

**Базовая цель проекта.**

Предсказать вероятность покупки в течение 90 дней.

## Загрузка данных.

### Загрузка библиотек.

In [ ]:
try:
    %pip install catboost -q
    %pip install phik -q
    %pip install --upgrade scikit-learn==1.3.2 -q # scikit-learn==1.1.3 -q
    %pip install --upgrade seaborn -q
    %pip install --upgrade numba -q
    %pip install shap -q
except Exception as error:
    print(f'Ошибка загрузки: {error}')

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
try:
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    import re
    import seaborn as sns
    import shap
    import warnings
    from catboost import CatBoostClassifier
    from joblib import dump
    from scipy import stats as st
    from phik import phik_matrix
    from phik.report import plot_correlation_matrix
    from sklearn.compose import ColumnTransformer
    from sklearn.dummy import DummyClassifier, DummyRegressor
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.impute import SimpleImputer
    from sklearn.linear_model import LinearRegression, LogisticRegression
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
    from sklearn.svm import SVC
    from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
    from sklearn.metrics import (
        accuracy_score,
        confusion_matrix,
        roc_auc_score,
        recall_score,
        precision_score,
        f1_score)
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import (
        OneHotEncoder,
        OrdinalEncoder,
        LabelEncoder,
        MinMaxScaler,
        StandardScaler)
except ImportError as error:
    print(f'Ошибка импорта: {error}')

### Константы.

In [ ]:
ZERO = 0
ONE = 1
TWO = 2
THREE = 3
SIX = 6
TEN = 10
ID = 'bulk_campaign_id'
RANDOM_STATE = 42
TEST_SIZE = 0.25
COLOR_ONE = 'SteelBlue'
COLOR_TWO = 'Yellow'
COLOR_THREE = 'Green'
COLOR_FOUR = 'Blues'
PATH = 'datasets/'

### Загрузка данных из csv-файлов в датафрейм.

In [ ]:
try:
    messages = pd.read_csv(f'{PATH}apparel-messages.csv')
    purchases = pd.read_csv(f'{PATH}apparel-purchases.csv')
    target = pd.read_csv(f'{PATH}apparel-target_binary.csv')
    event_chanel = pd.read_csv(f'{PATH}full_campaign_daily_event_channel.csv')
    event = pd.read_csv(f'{PATH}full_campaign_daily_event.csv')
except FileNotFoundError:
    print(f'Что то пошло не так: {error}')

### Дополнительные настройки.

In [ ]:
# Отключаем предупреждения.
pd.options.mode.chained_assignment = None

# Задаем два знака после запятой для чисел с плавающей запятой.
pd.options.display.float_format = '{:,.2f}'.format

# Игнорирование предупреждений.
warnings.filterwarnings("ignore")

# Установка опции для отображения максимальной ширины столбца.
pd.set_option('display.max_colwidth', None)

### Вспомогательные методы.

In [ ]:
# Добавим имена датафреймов в список.
# data_frames = {'messages': messages, 
#                'purchases': purchases,
#                'target': target,
#                'event_chanel': event_chanel,
#                'event': event}
df_names = ['messages', 'purchases', 'target', 'event_chanel', 'event']

In [ ]:
# Кастомный метод вывода информации о датафрейме.
def custom_info(value,
                name='',
                dtype_recom=False,
                is_result=True,
                sample=False,
                sample_quantity=TEN,
                head_value=0):
    result = (
        pd.DataFrame(value.count(),
                     columns=['non_null_count']))
    result['dtype'] = (
        pd.DataFrame(value.dtypes, 
                     columns=['dtype']))

    if name:
        print(f'Статистика по столбцам, выборка {name}.')

    if dtype_recom:
        result['dtype_recomended'] = (
            pd.DataFrame(
                value.convert_dtypes()
                .dtypes.to_frame()
                .rename(columns={0: 'dtype_recomended'}), 
                        columns=['dtype_recomended']))

    if is_result:
        display(result)
    print(f'Количество записей: {len(value)}')
    print(f'Количество столбцов: {len(value.columns)}')

    if sample:
        display(value.sample(sample_quantity).T)

    if head_value != 0:
        display(value.head(head_value).T)

In [ ]:
# Проверка пустых значений в датафрейме.
def count_none_values_table(value, name=''):
    try:
        result = (
            (value.isna().sum())
            .to_frame()
            .rename(columns={0: 'count_none'})
            .query('count_none > 0'))
        if len(result) != 0:
            print(f'Пропущенные значения в датафрейме {name}.'.strip())
            display(result.sort_values(by='count_none', ascending=True))
        else:
            print(f'В датафрейме {name} нет пропущенных значений.'
                  .replace('  ', ' '))
    except Exception as error:
        print(f'Что пошло не так: {error}')

### Общая информация о полученных датафреймах.

Посмотрим на статистику по столбцам.<br/>
Выведем на экран по 10 случайных позиций данных из каждого датафрейма. Посмотрим на их состав.

- 'messages': messages
- 'purchases': purchases
- 'target': target
- 'event_chanel': event_chanel
- 'event': event

In [ ]:
for name in df_names:
    if name in locals():
        custom_info(locals()[name], name, sample=True)
    else:
        print(f'Датафрейм {name} не найден!')

Статистика по столбцам, выборка messages.


,non_null_count,dtype
bulk_campaign_id,12739798,int64
client_id,12739798,int64
message_id,12739798,object
event,12739798,object
channel,12739798,object
date,12739798,object
created_at,12739798,object


Количество записей: 12739798
Количество столбцов: 7


,9931612,2742053,1376290,1845397,6566291,2995095,11588321,563771,10924438,5251958
bulk_campaign_id,14241,8169,6223,6963,13567,8504,14484,5231,14399,12838
client_id,1515915625832289612,1515915625490315837,1515915625491932090,1515915625486277447,1515915625577975272,1515915625490287770,1515915625646128135,1515915625752391496,1515915625468137509,1515915625557867983
message_id,1515915625832289612-14241-656ef61cca905,1515915625490315837-8169-6381ebaf63bce,1515915625491932090-6223-630c9088b14fd,1515915625486277447-6963-633fcb270a8d3,1515915625577975272-13567-64c3c130921aa,1515915625490287770-8504-6392e452ed99d,1515915625646128135-14484-65a637c671ca7,1515915625752391496-5231-62bdac6f371aa,1515915625468137509-14399-658bfce2abc43,1515915625557867983-12838-646efa6bbaaa3
event,send,send,open,send,open,send,send,send,open,open
channel,email,mobile_push,email,email,mobile_push,email,mobile_push,mobile_push,mobile_push,mobile_push
date,2023-12-05,2022-11-26,2022-09-01,2022-10-07,2023-07-28,2022-12-09,2024-01-16,2022-06-30,2023-12-27,2023-05-25
created_at,2023-12-05 10:40:26,2022-11-26 10:44:38,2022-09-01 16:31:03,2022-10-07 07:12:21,2023-07-28 14:48:28,2022-12-09 09:56:30,2024-01-16 09:00:38,2022-06-30 14:08:32,2023-12-27 11:54:07,2023-05-25 08:49:16


Статистика по столбцам, выборка purchases.


...

,56961,12872,25545,5331,104693,24492,57335,82679,75073,87849
date,2023-02-12,2022-08-02,2022-10-04,2022-06-22,2024-01-24,2022-09-29,2023-02-13,2023-08-09,2023-05-24,2023-10-02
bulk_campaign_id,3659,2499,1589,4517,14450,1625,7932,12367,9479,849
count_click_email,1,2,0,7,0,0,0,1,0,1
count_click_mobile_push,0,0,0,0,38,0,0,0,0,0
count_open_email,12,53,5,297,0,4,1,0,25,0
count_open_mobile_push,0,0,0,0,0,0,0,0,0,0
count_purchase_email,0,0,0,0,0,0,0,0,0,0
count_purchase_mobile_push,0,0,0,0,0,0,0,0,0,0
count_soft_bounce_email,0,0,0,0,0,0,0,0,0,0
count_subscribe_email,0,0,0,0,0,0,0,0,0,0


Статистика по столбцам, выборка event.


,non_null_count,dtype
date,131072,object
bulk_campaign_id,131072,int64
count_click,131072,int64
count_complain,131072,int64
count_hard_bounce,131072,int64
count_open,131072,int64
count_purchase,131072,int64
count_send,131072,int64
count_soft_bounce,131072,int64
count_subscribe,131072,int64


Количество записей: 131072
Количество столбцов: 24


,18810,81086,34557,52027,66002,91552,126306,71016,104231,126373
date,2022-09-01,2023-07-18,2022-11-13,2023-01-24,2023-04-03,2023-11-06,2024-04-26,2023-05-01,2024-01-22,2024-04-26
bulk_campaign_id,5201,13490,6316,5080,11436,14057,13852,11766,14279,14624
count_click,0,61,1,0,12,2,0,0,0,1
count_complain,0,0,0,543,0,0,0,0,0,0
count_hard_bounce,0,0,0,0,0,0,0,0,0,0
count_open,24,1670,86,5,0,17,3,1,3,65
count_purchase,0,0,0,0,0,0,0,0,0,0
count_send,0,0,0,0,0,0,0,0,0,0
count_soft_bounce,0,0,0,0,0,0,0,0,0,0
count_subscribe,0,1,0,0,0,0,0,0,0,0


#### Проверим наличие пропусков в датафреймах.

In [ ]:
for name in df_names:
    if name in locals():
        count_none_values_table(locals()[name], name)
    else:
        print(f'Датафрейм {name} не найден!')

В датафрейме messages нет пропущенных значений.
В датафрейме purchases нет пропущенных значений.
В датафрейме target нет пропущенных значений.
В датафрейме event_chanel нет пропущенных значений.
В датафрейме event нет пропущенных значений.


По результату изучения общей информации о каждом датафрейме, наблюдаем следующую картину:
* Наборы данных далее датафреймы с характеристиками сотрудников компании, содержать данные о тренировочной выборке; входных признаков тестовой выборки; целевого признака тестовой выборки.
* Типы данных у каждого признака соответствуют корректным значениям. 
* В датафреймах отсутствуют пропущенные значения, количество ненулевых значений соотвествует количеству записей в датафреймах.

###  Предобработка данных.

#### Исследование датафреймов на явное дублирование данных.

In [ ]:
for name in df_names:
    if name in locals():
        print(f'Количество явных дубликатов в датафрейме {name} - ', end='')
        print(locals()[name].duplicated().sum())
    else:
        print(f'Датафрейм {name} не найден!')

Количество явных дубликатов в датафрейме messages - 48610
Количество явных дубликатов в датафрейме purchases - 73020
Количество явных дубликатов в датафрейме target - 0
Количество явных дубликатов в датафрейме event_chanel - 0
Количество явных дубликатов в датафрейме event - 0


#### Исследование датафреймов на неявное дублирование данных.

In [ ]:
# Создадим метод для проверки уникальных значений в столбцах.
def check_unique_value(data, name, type='object', cols=[], id=[]):
    # создаём список с категориями
    if not cols:
        cols = data.drop(columns=id).select_dtypes(include=type).columns.tolist() # .drop(columns=[ID])
    if cols:
        print(f'\nУникальные значения поля(ей) в датафрейме {name}:')
        for value in cols:
            print(f'{value:16} - {data[value].unique().tolist()}')   
        print(f'Размер датафрейма - {data.shape}')

bulk_campaign_id	12739798	int64
client_id	12739798	int64
message_id	12739798	object
event	12739798	object
channel	12739798	object
date	12739798	object
created_at	12739798	object


In [ ]:
# Проверим уникальные значения в текстовых столбцах датафрейма messages.
check_unique_value(messages, 'messages', cols=['event', 'channel'])


Уникальные значения поля(ей) в датафрейме messages:
event            - ['open', 'click', 'purchase', 'send', 'unsubscribe', 'hbq_spam', 'hard_bounce', 'subscribe', 'soft_bounce', 'complain', 'close']
channel          - ['email', 'mobile_push']
Размер датафрейма - (12739798, 7)
